# Project: Indian Startup Funding Analysis (2015–2024)

# ===============================================
#  Step 1: Import Required Libraries
# ===============================================

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [10]:
# Set display and visualization styles

pd.set_option('display.max_columns', None)
sns.set(style="whitegrid", palette="Set2")

In [11]:
# Load the dataset'

df = pd.read_csv("C:\\Users\\Pooja001\\OneDrive\\문서\\Data_Analytics_Projects\\Indian_Startup_Funding_Analysis\\Data\\startup_funding_2012_2024_raw.csv")
print("Dataset Loaded Successfully")
print("Shape of Dataset:", df.shape)
df.head()

Dataset Loaded Successfully
Shape of Dataset: (3054, 10)


,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09-01-2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13-01-2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09-01-2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02-01-2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02-01-2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


# ===============================================
#  Step 2: Initial Data Cleaning
# ===============================================

In [12]:
# Rename columns for consistency

df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace("-", "_")

In [13]:
# Check for duplicates

print("Duplicate Rows:", df.duplicated().sum())

Duplicate Rows: 0


In [14]:
# Drop Duplicates

df.drop_duplicates(inplace = True)

In [15]:
# Removing leading / Trailing spaces in String Columns

df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [16]:
# Convert Date Column to DateTime Format

df['Date_dd/mm/yyyy'] = pd.to_datetime(df['Date_dd/mm/yyyy'], errors = 'coerce', dayfirst = True)
df.rename(columns={'Date_dd/mm/yyyy': 'Date'}, inplace=True)

In [17]:
# Clean Amount in USD - Remove commas, missing values, convert to float

df['Amount_in_USD'] = (df['Amount_in_USD'].astype(str).str.replace(r'[^\d.]','', regex = True).replace('', np.nan).astype(float))

In [18]:
# Count nulls BEFORE handling (audit)

null_summary = df.isna().sum().sort_values(ascending = False)
print("Null counts before handling:\n", null_summary)

Null counts before handling:
 Remarks              2625
Amount_in_USD         967
SubVertical           936
City__Location        180
Industry_Vertical     171
Investors_Name         24
Date                    8
InvestmentnType         4
Sr_No                   0
Startup_Name            0
dtype: int64


In [19]:
# Drop rows with no startup name or year (critical data)

df.dropna(subset=['Startup_Name'], inplace=True)
df = df[~df['Date'].isna()] 

In [20]:
# Extract Year and Month

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month_name()

In [21]:
# Handle categorical/text columns: fill with 'Unknown' or 'Undisclosed'
fill_unknown_cols = ['Industry_Vertical', 'SubVertical', 'City__Location', 
                     'Investors_Name', 'InvestmentnType', 'Remarks']

mapped = {}
for col in fill_unknown_cols:
    # find best matching actual column name (loose)
    candidates = [c for c in df.columns if c.lower().replace(' ', '').startswith(col.lower().replace('_','').replace(' ','')[:6])]
    mapped[col] = candidates[0] if candidates else None

for key, actual in mapped.items():
    if actual and actual in df.columns:
        df[actual] = df[actual].fillna('Unknown')


In [24]:
# Impute Amount_in_USD intelligently

global_median = df['Amount_in_USD'].median(skipna = True)

In [25]:
# pick existence columns for grouping

industry_col = None
for c in df.columns:
    if 'industry' in c.lower():
        industry_col = c
        break

if industry_col is None:
    # fallback: create a placeholder industry column if missing
    df['Industry_Vertical'] = 'Unknown'
    industry_col = 'Industry_Vertical'

In [26]:
# median by industry+year

median_ind_year = df.groupby([industry_col, 'Year'])['Amount_in_USD'].median().reset_index().rename(columns={'Amount_in_USD':'median_ind_year'})
median_ind = df.groupby(industry_col)['Amount_in_USD'].median().reset_index().rename(columns={'Amount_in_USD':'median_ind'})

In [27]:
# Merge medians back to df for imputation

df = df.merge(median_ind_year, on=[industry_col, 'Year'], how='left')
df = df.merge(median_ind, on=industry_col, how='left')

In [28]:
def impute_amount(row):
    if pd.notnull(row['Amount_in_USD']):
        return row['Amount_in_USD']
    if pd.notnull(row.get('median_ind_year')):
        return row['median_ind_year']
    if pd.notnull(row.get('median_ind')):
        return row['median_ind']
    return global_median

df['Amount_in_USD'] = df.apply(impute_amount, axis=1)

In [29]:
# Post-imputation: keep audit info - mark rows that were originally missing amount

df['Amount_was_missing'] = df['Amount_in_USD'].isna()  # before imputation would have been True; but we already imputed, so set flag earlier would've been better

# To preserve original missing flag we can reconstruct:
# original_missing_flag = df['median_ind_year'].isna() & df['median_ind'].isna() & (df['Amount_in_USD'].isna())
# But since we've replaced, instead create a flag where Amount is equal to medians or global median.
df['Imputed_From'] = np.where(
    df['Amount_in_USD'] == df['median_ind_year'], 'median_ind_year',
    np.where(df['Amount_in_USD'] == df['median_ind'], 'median_ind',
             np.where(df['Amount_in_USD'] == global_median, 'global_median', 'original')
    )
)

In [31]:
# Standardize city names if present
city_col = None
for c in df.columns:
    if 'city' in c.lower():
        city_col = c
        break

if city_col:
    df[city_col] = df[city_col].replace({
        'Bangalore': 'Bengaluru',
        'Bengaluru ': 'Bengaluru',
        'Delhi NCR': 'Delhi',
        'Gurgaon': 'Gurugram',
        'Bombay': 'Mumbai'
    }).fillna('Unknown')
    df.rename(columns={city_col: 'City_Location'}, inplace=True)
else:
    df['City_Location'] = 'Unknown'

In [32]:
# Recompute 'No_of_Investors' robustly

inv_col = None
for c in df.columns:
    if 'investor' in c.lower():
        inv_col = c
        break

if inv_col:
    df['Investors_Name'] = df[inv_col].fillna('Unknown')
else:
    df['Investors_Name'] = 'Unknown'

In [33]:
# Count commas as separators; if 'Unknown' -> 0

def count_investors(text):
    if pd.isna(text) or text.strip().lower() in ['unknown', 'undisclosed', 'nan']:
        return 0
    # split by common separators
    parts = [p.strip() for p in re.split(r',|;| and |&', text) if p.strip()]
    return len(parts)

import re
df['No_of_Investors'] = df['Investors_Name'].apply(count_investors)

In [34]:
# Final cleanup: drop helper median columns

df.drop(columns=['median_ind_year', 'median_ind'], errors='ignore', inplace=True)

In [35]:
# 14) Final null summary after handling

null_summary_after = df.isna().sum().sort_values(ascending=False)
print("Null counts after handling:\n", null_summary_after)

Null counts after handling:
 InvestmentnType       4
Date                  0
Sr_No                 0
Industry_Vertical     0
SubVertical           0
City_Location         0
Startup_Name          0
Investors_Name        0
Amount_in_USD         0
Remarks               0
Year                  0
Month                 0
Amount_was_missing    0
Imputed_From          0
No_of_Investors       0
dtype: int64


In [36]:
df.head()

,Sr_No,Date,Startup_Name,Industry_Vertical,SubVertical,City_Location,Investors_Name,InvestmentnType,Amount_in_USD,Remarks,Year,Month,Amount_was_missing,Imputed_From,No_of_Investors
0,1,2020-01-09,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,200000000.0,Unknown,2020,January,False,median_ind_year,1
1,2,2020-01-13,Shuttl,Transportation,App based shuttle service,Gurugram,Susquehanna Growth Equity,Series C,8048394.0,Unknown,2020,January,False,median_ind_year,1
2,3,2020-01-09,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,18358860.0,Unknown,2020,January,False,median_ind_year,1
3,4,2020-01-02,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,3000000.0,Unknown,2020,January,False,median_ind_year,1
4,5,2020-01-02,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,1800000.0,Unknown,2020,January,False,median_ind_year,1


In [40]:
# Save cleaned file (audit-ready)

df.to_csv("C:\\Users\\Pooja001\\OneDrive\\문서\\Data_Analytics_Projects\\Indian_Startup_Funding_Analysis\\Data\\cleaned_startup_funding_2012_2024_raw.csv", index=False)

print("Cleaning complete. Rows:", df.shape[0])
print('Final Shape:', df.shape)

Cleaning complete. Rows: 3046
Final Shape: (3046, 15)
